In [ ]:
# default_exp vulnerabilidad

In [ ]:
%load_ext autoreload
%autoreload 2

# Vulnerabilidad

> Métodos para dignosticar la vulnerabilidad a COVID-19 de entidades y municipios basandonos en sus 
indicadores sociales, económicos, de salud y de infraestructura

In [ ]:
# exports
from ipywidgets import IntProgress
from IPython.display import display
import time

from covid19_vulnerabilidad_mex.datos import *

import pandas as pd
import geopandas as gpd

import glob
import os
import matplotlib.pyplot as plt
import numpy as np

from datetime import timedelta
import datetime

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_decomposition import PLSRegression

import imageio
from pygifsicle import optimize

## Exploración visual

### show_feature_importances

In [ ]:
# exports

def show_feature_importances(rf):
    importances = rf.feature_importances_

    std = np.std([tree.feature_importances_ for tree in rf.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))

    # Plot the feature importances of the forest
    f, ax = plt.subplots(figsize=(15, 10))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), X.columns[indices])
    plt.xlim([-1, X.shape[1]])
    plt.show()

In [ ]:
# exports

def mostrar_coeficientes_PLS(pls):
    f, ax = plt.subplots(figsize=(15, 10))
    sns.barplot(y='nombre', x='coef', data=feats_df, color='c')

## Preparación de datos

## agregar_conteo_pruebas

In [ ]:
# exports

def agregar_conteo_pruebas(covid_municipal, solo_covid=True):
    cols_localidad = ['ENTIDAD_RES', 'MUNICIPIO_RES', 'CLAVE_MUNICIPIO_RES']
    count_tested = covid_municipal[cols_localidad + ['conteo']].groupby(cols_localidad).sum()
    count_tested.reset_index(inplace=True)
    count_tested.rename(columns={'conteo': 'total_pruebas'}, inplace=True)

    covid_municipal = covid_municipal.merge(count_tested, on=cols_localidad, how='left')


    covid_municipal['casos_frac'] = 100 * covid_municipal['conteo'] / covid_municipal['total_pruebas']
    covid_municipal['tasa_covid_letal'] = 100 * covid_municipal['defunciones'] / covid_municipal['conteo']

    if solo_covid:
        covid_municipal = covid_municipal.query('RESULTADO == "Positivo SARS-CoV-2"')
    return covid_municipal

## agregar_tasas_municipales

In [ ]:
# exports

def agregar_tasas_municipales(casos_mun_df):
    casos_mun_df['covid_confirmados_100k'] = 100000 * casos_mun_df['conteo'] / casos_mun_df['pob2020']
    casos_mun_df['covid_defun_100k'] = 100000 * casos_mun_df['defunciones'] / casos_mun_df['pob2020']
    casos_mun_df['tasa_covid_letal'] = 100 * casos_mun_df['defunciones'] / casos_mun_df['conteo']

    # covid_municipal = covid_municipal.query('RESULTADO == "Positivo SARS-CoV-2"').copy()

    # casos_mun_df = gpd.GeoDataFrame(casos_mun_df, geometry='geometry')
    
    return casos_mun_df

In [ ]:
fecha = '200601'
covid_municipal = tabla_covid_indicadores_municipales(fecha)

In [ ]:
covid_municipal = agregar_tasas_municipales(covid_municipal)

In [ ]:
covid_municipal

## Ajuste de modelos de inferencia

### caracteristicas_modelos_municipios

In [ ]:
# exports

def caracteristicas_modelos_municipios(mun_df, poblaciones=False, i_vuln=False):
    pob_vars = []
    if not poblaciones:
        pob_vars = list(mun_df.columns[mun_df.columns.str.contains('_pob')])
        pob_vars = pob_vars + ['mayores_65', 'pob2020', 'pt_2015', 'pob_menore', 'poblacion', 'pob_total']
        pob_vars = pob_vars + ['sin_dere_1', 'sin_derech', 'carencias_']
    i_vuln_vars = []
    if not i_vuln:
        i_vuln_vars = ['i_vuln_salud', 'i_vuln_cobertura', 'i_vuln_econo', 'i_vuln_social',
                       'i_vuln_gen', 'i_vuln_infraestructura']
        
    columnas_numericas = mun_df.select_dtypes(include=np.number).columns
    
    otras_vars = ['covid_defun_100k', 'tasa_covid_letal', 'defunciones',
                  'INTUBADO_BIN', 'UCI_BIN', 'UCIs', 'tasa_uci', 'total_pruebas',
                  'area_cart', 'area', 'densi', 'casos_frac', 'area_km2', 'conteo',
                  'oid', 'covid_confrimados_100k', 'id', 'vulnerabilidad_ambiental_num',
                  'covid_confirmados_100k', 'index']
    
    caracteristicas = list(set(columnas_numericas).difference(pob_vars + i_vuln_vars + otras_vars))
 
    return caracteristicas
    

In [ ]:
caracteristicas_mun = caracteristicas_modelos_municipios(covid_municipal)
caracteristicas_mun

### ajustar_pls_letalidad

In [ ]:
#exports

def ajustar_pls_letalidad(municipios_df, caracteristicas, min_casos=20, min_defunciones=0):
    data_train = municipios_df.loc[municipios_df[caracteristicas].notna().all(axis=1)]

    X = data_train.query(f'(conteo > {min_casos}) & (defunciones > {min_defunciones})')[caracteristicas]
    Y = data_train.query(f'(conteo > {min_casos}) & (defunciones > {min_defunciones})')['tasa_covid_letal']

    # X['i_vuln_econo'] = -X['i_vuln_econo']

    pls2 = PLSRegression(n_components=1)
    pls2.fit(X, Y)
    pls2.coef_ = pls2.coef_.flatten()
    
    return pls2

In [ ]:
pls = ajustar_pls_letalidad(municipios_df=covid_municipal, caracteristicas=caracteristicas_mun)

### ajustar_pls_columna

In [ ]:
#exports

def ajustar_pls_columna(municipios_df, caracteristicas, columna, min_casos=20, min_defunciones=0):
    data_train = municipios_df.loc[municipios_df[caracteristicas].notna().all(axis=1)]

    X = data_train.query(f'(conteo > {min_casos}) & (defunciones > {min_defunciones})')[caracteristicas]
    try:
        Y = data_train.query(f'(conteo > {min_casos}) & (defunciones > {min_defunciones})')[columna]
    except KeyError:
        print(f"No existe la columna {columna}")
    # X['i_vuln_econo'] = -X['i_vuln_econo']

    pls2 = PLSRegression(n_components=1)
    pls2.fit(X, Y)
    pls2.coef_ = pls2.coef_.flatten()
    
    return pls2

In [ ]:
pls = ajustar_pls_columna(municipios_df=covid_municipal,
                          caracteristicas=caracteristicas_mun,
                          columna='tasa_covid_letal')


### ajustar_rf_letalidad

In [ ]:
# exports

def ajustar_rf_letalidad(municipios_df, caracteristicas, min_casos=20, min_defunciones=0, random_seed=0):
    data_train = municipios_df.loc[municipios_df[caracteristicas].notna().all(axis=1)]

    X = data_train.query(f'(conteo > {min_casos}) & (defunciones > {min_defunciones})')[caracteristicas]
    Y = data_train.query(f'(conteo > {min_casos}) & (defunciones > {min_defunciones})')['tasa_covid_letal']

    # X['i_vuln_econo'] = -X['i_vuln_econo']
    rf = RandomForestRegressor(random_state=random_seed)
    rf.fit(X, Y)

    return rf    

## Generar índices

### calificar_municipios_letalidad

In [ ]:
#exports

def calificar_municipios_letalidad(mun_df, regr, caracteristicas, etiqueta):
    not_na_row = mun_df[caracteristicas].notnull().all(axis=1)
    X = mun_df.loc[not_na_row, caracteristicas]
    Y_pred = regr.predict(X)
    mun_df.loc[not_na_row, etiqueta] = Y_pred
    
    return mun_df
    

### calificar_municipios_letalidad_formato_largo

In [ ]:
#exports

def calificar_municipios_letalidad_formato_largo(mun_df, regr, caracteristicas, modelo, dia_ajuste):
    mun_df = mun_df.copy()
    not_na_row = mun_df[caracteristicas].notnull().all(axis=1)
    X = mun_df.loc[not_na_row, caracteristicas]
    Y_pred = regr.predict(X)
    mun_df['modelo'] = modelo
    mun_df['dia_ajuste'] = dia_ajuste
    mun_df.loc[not_na_row, 'valor'] = Y_pred
    
    return mun_df
    

### seleccionar_caracteristicas

In [ ]:
#exports

def seleccionar_caracteristicas(regr, X, caracteristicas):
    sel = SelectFromModel(regr, prefit=True)

    caracteristicas_selec = list(X[caracteristicas].columns[sel.get_support()])
    return caracteristicas_selec

In [ ]:
seleccionar_caracteristicas(pls, covid_municipal, caracteristicas_mun)

### calcular_indices_vulnerabilidad

In [ ]:
# exports

def calcular_indices_vulnerabilidad(fecha_0, fecha_1, vulnerabilidad='fecha_0'):
    covid_municipal_0 = tabla_covid_indicadores_municipales(fecha_0)
    covid_municipal_0 = agregar_tasas_municipales(covid_municipal_0)
    
    covid_municipal_1 = tabla_covid_indicadores_municipales(fecha_1)
    covid_municipal_1 = agregar_tasas_municipales(covid_municipal_1)
    
    caracteristicas = caracteristicas_modelos_municipios(covid_municipal_0)    
    
    resultados = covid_municipal_1.copy()
    
    # rf = ajustar_rf_municipios(covid_municipal_0, fecha_0, caracteristicas)
    # caracteristicas_rf = seleccionar_caracteristicas(rf, covid_municipal_0, caracteristicas)
    # resultados = calificar_municipios_letalidad(resultados, rf, caracteristicas, etiqueta='i_RF_0')
    
    pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas)
    resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_0')
    
    pls = ajustar_pls_letalidad(covid_municipal_1, caracteristicas)
    resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_1')
    
    # pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas_rf)
    # resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_RF_1')
    
    return resultados


In [ ]:
evaluacion = calcular_indices_vulnerabilidad('200501', '200603')

In [ ]:
evaluacion

In [ ]:
evaluacion[evaluacion.i_PLS_0.notna()].plot(column='i_PLS_0', cmap='RdYlBu_r', scheme="Quantiles",
         figsize=(15, 20), legend=True, k=3)

In [ ]:
evaluacion[evaluacion.tasa_covid_letal.notna()].plot(column='tasa_covid_letal', cmap='RdYlBu_r', scheme="EqualInterval",
         figsize=(15, 20), legend=True, k=10)

In [ ]:
evaluacion[evaluacion.i_PLS_1.notna()].plot(column='i_PLS_1', cmap='RdYlBu_r', scheme="Quantiles",
         figsize=(15, 20), legend=True, k=3)

### calcular_indices_vulnerabilidad_formato_largo

In [ ]:
# exports

def calcular_indices_vulnerabilidad_formato_largo(fecha_0, fecha_1):
    covid_municipal_0 = tabla_covid_indicadores_municipales(fecha_0)
    covid_municipal_0 = agregar_tasas_municipales(covid_municipal_0)
    
    covid_municipal_1 = tabla_covid_indicadores_municipales(fecha_1)
    covid_municipal_1 = agregar_tasas_municipales(covid_municipal_1)
    
    caracteristicas = caracteristicas_modelos_municipios(covid_municipal_0)    
    
    # rf = ajustar_rf_municipios(covid_municipal_0, fecha_0, caracteristicas)
    # caracteristicas_rf = seleccionar_caracteristicas(rf, covid_municipal_0, caracteristicas)
    # resultados = calificar_municipios_letalidad(resultados, rf, caracteristicas, etiqueta='i_RF_0')
    
    pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas)
    resultados = calificar_municipios_letalidad_formato_largo(covid_municipal_0, pls, caracteristicas,
                                                modelo='PLS', dia_ajuste=fecha_0)
    
    pls = ajustar_pls_letalidad(covid_municipal_1, caracteristicas)
    resultados_temp = calificar_municipios_letalidad_formato_largo(covid_municipal_1, pls, caracteristicas,
                                                     modelo='PLS', dia_ajuste=fecha_1)
    
    resultados = pd.concat([resultados, resultados_temp], ignore_index=True)
    # pls = ajustar_pls_letalidad(covid_municipal_0, caracteristicas_rf)
    # resultados = calificar_municipios_letalidad(resultados, pls, caracteristicas, etiqueta='i_PLS_RF_1')
    
    return resultados


In [ ]:
vulnerabilidad_covid = calcular_indices_vulnerabilidad_formato_largo('200601', '200614')

### calcular_periodo_vulnerabilidad

In [ ]:
# exports

def calcular_periodo_vulnerabilidad(inicio, fin, min_defunciones=-1):
    inicio = pd.to_datetime(inicio, yearfirst=True)
    fin = pd.to_datetime(fin, yearfirst=True)
    
    fechas = pd.date_range(inicio, fin)
    resultados = []
    modelos = []
    
    asegura_archivos_covid_disponibles(fechas)

    f = IntProgress(min=0, max=len(fechas) - 1) # instantiate the bar
    display(f) # display the bar
    
    for count, fecha in enumerate(fechas):
        covid_municipal = tabla_covid_indicadores_municipales(fecha.strftime("%y%m%d"))
        covid_municipal = agregar_tasas_municipales(covid_municipal)
     
        caracteristicas = caracteristicas_modelos_municipios(covid_municipal)
        pls = ajustar_pls_letalidad(covid_municipal, caracteristicas, min_defunciones=min_defunciones)
        df = calificar_municipios_letalidad_formato_largo(covid_municipal, pls, caracteristicas,
                                                    modelo='PLS', dia_ajuste=fecha)
        resultados.append(df)
        modelo = pd.DataFrame({'caracteristica': caracteristicas, 'coef': pls.coef_})
        modelo['dia_ajuste'] = fecha
        modelo['modelo'] = 'PLS'
        modelos.append(modelo)
        
        rf = ajustar_rf_letalidad(covid_municipal, caracteristicas, min_defunciones=min_defunciones)
        df = calificar_municipios_letalidad_formato_largo(covid_municipal, rf, caracteristicas,
                                            modelo='RF', dia_ajuste=fecha)
        resultados.append(df)
        modelo = pd.DataFrame({'caracteristica': caracteristicas, 'coef': rf.feature_importances_})
        modelo['dia_ajuste'] = fecha
        modelo['modelo'] = 'RF'
        modelos.append(modelo)
        
        f.value = count
    
    resultados_df = pd.concat(resultados, ignore_index=True)
    modelos_df = pd.concat(modelos, ignore_index=True)
    return modelos_df, resultados_df

In [ ]:
modelos, serie_vulnerabilidad = calcular_periodo_vulnerabilidad('20-04-24', '20-06-02')

In [ ]:
serie_vulnerabilidad

In [ ]:
serie_vulnerabilidad[serie_vulnerabilidad.MUNICIPIO_RES == 'AGUASCALIENTES']

## calcular_periodo_vulnerabilidad con un sólo archivo de datos

### periodo_vulnerabilidad_con_dataframe

In [ ]:
# exports


def periodo_vulnerabilidad_con_dataframe(covid_municipal, inicio, fin, columna='tasa_covid_letal',
                                         min_casos=20, min_defunciones=-1, rf=True):
    """Calcula la vulnerabilidad (PLS) para todo el periodo usando como objetivo 
       la columna que se le pase.

    :param df: el dataframe con los datos para ajustar el modelo. Debe traer ya las tasas municipales
    :type df: pd.DataFrame
    :param inicio: fecha inicial (Y-m-d)
    :type inicio: str
    :param fin: fecha final (Y-m-d)
    :type fin: str
    :param columna: la columna para usar como objetivo, el default es 'tasa_covid_letal'
    :type columna: str
    :param min_casos: Número mínimo de casos para considerar a un municipio
    :type min_casos: int
    :param min_defunciones: Número mínimo de defunciones para considerar a un municipio
    :type min_defunciones: int
    :param rf: True/False ajustar también un nmodelo de Random Forest a los dato
    :type rf: bool

    :returns: Un DataFrame igual que el de entrada pero cun una columna extra con el resultado 
              del modelo. La columna se llama 'valor_{columna}'
    :rtype: gpd.GeoDataFrame

    """
    inicio = pd.to_datetime(inicio, yearfirst=True)
    fin = pd.to_datetime(fin, yearfirst=True)
    fin = min(covid_municipal.FECHA_INGRESO.max(), fin)
    fechas = pd.date_range(inicio, fin)
    resultados = []
    modelos = []
    f = IntProgress(min=0, max=len(fechas) - 1) # instantiate the bar
    display(f) # display the bar
    # covid_municipal = agregar_tasas_municipales(df)
    caracteristicas = caracteristicas_modelos_municipios(covid_municipal)
    for count, fecha in enumerate(fechas):
        covid_municipal_fecha = covid_municipal.query(
            f'FECHA_INGRESO == "{fecha.strftime("%Y-%m-%d")}"')
        pls = ajustar_pls_columna(covid_municipal_fecha,
                                    caracteristicas,
                                    columna=columna,
                                    min_casos=min_casos,
                                    min_defunciones=min_defunciones)
        df = calificar_municipios_letalidad_formato_largo(covid_municipal_fecha,
                                                          pls,
                                                          caracteristicas,
                                                          modelo='PLS', dia_ajuste=fecha)
        resultados.append(df)
        modelo = pd.DataFrame({'caracteristica': caracteristicas, 'coef': pls.coef_})
        modelo['dia_ajuste'] = fecha
        modelo['modelo'] = 'PLS'
        modelos.append(modelo)
        if rf:
            rf = ajustar_rf_letalidad(covid_municipal_fecha,
                                    caracteristicas,
                                    min_casos=min_casos,
                                    min_defunciones=min_defunciones)
            df = calificar_municipios_letalidad_formato_largo(covid_municipal_fecha,
                                                            rf, caracteristicas,
                                                            modelo='RF',
                                                            dia_ajuste=fecha)
            resultados.append(df)
            modelo = pd.DataFrame({'caracteristica': caracteristicas, 'coef': rf.feature_importances_})
            modelo['dia_ajuste'] = fecha
            modelo['modelo'] = 'RF'
            modelos.append(modelo)       
        f.value = count
    
    resultados_df = pd.concat(resultados, ignore_index=True)
    modelos_df = pd.concat(modelos, ignore_index=True)   
    resultados_df = gpd.GeoDataFrame(resultados_df, geometry='geometry')
    resultados_df.rename({'valor':f'valor_{columna}'}, axis=1, inplace=True)
    return modelos_df, resultados_df
    
def calcular_periodo_vulnerabilidad_2(fecha_archivo, inicio, fin, min_casos=10, min_defunciones=-1):
    asegura_archivos_covid_disponibles([fecha_archivo])  
    covid_municipal = serie_covid_indicadores_municipales(fecha_archivo.strftime("%y%m%d"))
    
    modelos_df, resultados_df = periodo_vulnerabilidad_con_dataframe(covid_municipal,
                                                                     inicio,
                                                                     fin,
                                                                     min_casos=min_casos,
                                                                     min_defunciones=min_defunciones)
    
    return modelos_df, resultados_df


In [ ]:
covid_municipal = serie_covid_indicadores_municipales('200830')

In [ ]:
modelos, resultados = periodo_vulnerabilidad_con_dataframe(covid_municipal, '20-08-01', '20-08-15')

In [ ]:
modelos.head()

,caracteristica,coef,dia_ajuste,modelo
0,irs_2015,0.035450,2020-08-01,PLS
1,tasa_cardiacas,-0.054922,2020-08-01,PLS
2,hospitales_imss,-0.022300,2020-08-01,PLS
3,total_de_camas_privados,-0.029194,2020-08-01,PLS
4,vpt_2015,-0.005140,2020-08-01,PLS


In [ ]:
resultados.head()

,index,ENTIDAD_RES,MUNICIPIO_RES,CLAVE_MUNICIPIO_RES,RESULTADO,FECHA_INGRESO,conteo,defunciones,pt_2015,an_2015,ne614_015,bi_2015,sins_15,vpt_2015,vns_2015,vna_2015,vnd_2015,vne_2015,vnl_2015,vnr_2015,irs_2015,grs_2015,lmex_2015,poblacion,pobreza,pobreza_pob,pobreza_e,pobreza_e_pob,pobreza_m,pobreza_m_pob,vul_car,vul_car_pob,vul_ing,vul_ing_pob,npnv,npnv_pob,ic_rezedu,ic_rezedu_pob,ic_asalud,ic_asalud_pob,ic_segsoc,ic_segsoc_pob,ic_cv,ic_cv_pob,ic_sbv,ic_sbv_pob,ic_ali,ic_ali_pob,carencias,carencias_pob,carencias3,carencias3_pob,plb,plb_pob,plbm,plbm_pob,pob_total,mayores_65,pob_menore,sin_derech,sin_dere_1,consultori,consulto_1,farmacias,porc_carencia_salud,tasa_diabetes,tasa_cardiacas,tasa_cancer,tasa_pulmonares,grado_diabetes,grado_cardiacas,grado_cancer,grado_pulmonares,hospitales_imss,hospitales_issste,hospitales_ssa,hospitales_sme,hospitales_semar,hospitales_sedena,hospitales_pemex,total_hospitales_publicos,total_camas_publicos,hospitales_privados,total_de_camas_privados,total_camas,pob2020,geometry,covid_confirmados_100k,covid_defun_100k,tasa_covid_letal,modelo,dia_ajuste,valor
0,430.0,AGUASCALIENTES,AGUASCALIENTES,01001,Positivo SARS-CoV-2,2020-08-01,3214.0,280.0,877190,2.0570,3.4786,25.6873,14.1560,0.5478,0.5543,0.6627,0.4819,0.1999,13.8229,5.4216,-1.365841,Muy bajo,2397.0,861446.0,26.1,224949.0,1.6,13650.0,24.5,211299.0,26.4,227805.0,11.5,98641.0,36.0,310052.0,11.7,100530.0,15.1,129728.0,36.4,313557.0,3.7,32153.0,3.0,25494.0,18.0,154658.0,52.6,452753.0,7.3,63220.0,37.6,323590.0,10.0,86166.0,797010,40309,169531,178471,304855,575,23,656,15.1,0.537007,1.273510,0.749050,0.599742,Bajo,Bajo,Medio,Medio,2,1,5,0,0,0,0,8,1228,17,43,1271,961977,"POLYGON ((-102.09775 22.02325, -102.11598 22.0...",334.103622,29.106725,8.711886,PLS,2020-08-01,10.508968
1,1117.0,AGUASCALIENTES,ASIENTOS,01002,Positivo SARS-CoV-2,2020-08-01,72.0,6.0,46464,4.4207,2.5959,41.8921,5.5355,1.6779,5.6457,1.2979,5.9887,1.1681,23.4356,15.2220,-0.883915,Bajo,1950.0,46624.0,54.0,25169.0,4.4,2067.0,49.5,23101.0,29.0,13544.0,7.7,3577.0,9.3,4335.0,19.8,9244.0,6.0,2788.0,69.0,32175.0,7.8,3633.0,16.2,7553.0,24.7,11494.0,83.0,38713.0,14.1,6594.0,61.7,28745.0,24.4,11394.0,45492,2627,11666,6264,34652,10,1,24,6.0,0.791348,0.989185,0.637475,0.769366,Medio,Muy Bajo,Medio,Alto,0,0,0,0,0,0,0,0,0,0,0,0,50864,"POLYGON ((-102.04348 22.29392, -102.05861 22.3...",141.553948,11.796162,8.333333,PLS,2020-08-01,13.521364
2,1804.0,AGUASCALIENTES,CALVILLO,01003,Positivo SARS-CoV-2,2020-08-01,125.0,3.0,56048,4.7805,4.3906,49.1856,9.7524,1.0577,0.7756,0.9519,0.8673,0.5429,15.5620,7.3826,-0.979457,Bajo,2077.0,52773.0,56.8,29951.0,3.1,1650.0,53.6,28301.0,27.6,14561.0,6.3,3348.0,9.3,4913.0,23.2,12254.0,9.9,5212.0,73.6,38857.0,5.2,2746.0,7.7,4082.0,17.3,9148.0,84.3,44513.0,10.5,5528.0,63.1,33299.0,26.8,14118.0,54136,4034,12803,9263,45895,26,3,32,9.9,0.923600,1.293040,0.738880,0.664992,Medio,Bajo,Medio,Alto,0,0,1,0,0,0,0,1,46,1,0,46,60760,"POLYGON ((-102.57625 21.96778, -102.59749 22.0...",205.727452,4.937459,2.400000,PLS,2020-08-01,13.514759
3,2491.0,AGUASCALIENTES,COSÍO,01004,Positivo SARS-CoV-2,2020-08-01,67.0,0.0,15577,4.3233,2.6265,33.0597,5.1165,1.5629,2.8986,0.5968,1.9324,0.7389,23.0747,12.4467,-1.062688,Muy bajo,2164.0,15321.0,43.2,6620.0,2.8,432.0,40.4,6187.0,32.7,5004.0,8.0,1232.0,16.1,2466.0,14.7,2251.0,5.5,846.0,62.7,9608.0,7.9,1215.0,7.6,1167.0,20.7,3175.0,75.9,11623.0,9.9,1522.0,51.2,7852.0,16.6,2546.0,15042,844,3651,2092,11093,2,0,8,5.5,0.797766,1.595533,0.797766,0.664805,Medio,Medio,Alto,Alto,0,0,0,0,0,0,0,0,0,0,0,0,16918,"POLYGON ((-102.26605 22.40372, -102.28993 22.4...",396.027899,0.000000,0.000000,PLS,2020-08-01,12.856164
4,3865.0,AGUASCALIENTES,JESÚS MARÍA,01005,Positivo SARS-CoV-2,2020-08-01,143.0,7.0,120405,3.2445,4.1297,33.7271,13.8341,0.9201,1.0273,0.9443,0.7264,0.4220,15.5097,8.1976,-1.188620,Muy bajo,2286.0,125750.0,25.0,31479.0,1.7,2095.0,23.4,29385.0,38.0,47831.0,6.6,8314.0,30.3,38126.0,17.3,21712.0,15.2,19076.0,45.0,56595.0,8.2,10303.0,6.3,7977.0,18.7,23562.0,6

### agregar_periodo_vulnerabilidad

In [ ]:
#exports

def agregar_periodo_vulnerabilidad(modelos_df, serie_vulnerabilidad, inicio, fin):
    inicio = pd.to_datetime(inicio, yearfirst=True)
    fin = pd.to_datetime(fin, yearfirst=True)
    fechas = pd.date_range(inicio, fin)
    resultados = []
    modelos = []
    
    asegura_archivos_covid_disponibles(fechas)
    f = IntProgress(min=0, max=len(fechas) - 1) # instantiate the bar
    display(f) # display the bar
    
    for count, fecha in enumerate(fechas):
        covid_municipal = tabla_covid_indicadores_municipales(fecha.strftime("%y%m%d"))
        covid_municipal = agregar_tasas_municipales(covid_municipal)
     
        caracteristicas = caracteristicas_modelos_municipios(covid_municipal)
        pls = ajustar_pls_letalidad(covid_municipal, caracteristicas)
        df = calificar_municipios_letalidad_formato_largo(covid_municipal, pls, caracteristicas,
                                                    modelo='PLS', dia_ajuste=fecha)
        resultados.append(df)
        modelo = pd.DataFrame({'caracteristica': caracteristicas, 'coef': pls.coef_})
        modelo['dia_ajuste'] = fecha
        modelos.append(modelo)
        f.value = count
    
    resultados_df = pd.concat(resultados, ignore_index=True)    
    nuevos_modelos_df = pd.concat(modelos, ignore_index=True)
    
    serie_vulnerabilidad = serie_vulnerabilidad.append(resultados_df)
    modelos_df = modelos_df.append(nuevos_modelos_df)
    
    return modelos_df, serie_vulnerabilidad


### calcular_periodo_vulnerabilidad_urbanos

In [ ]:
# exports
def calcular_vulnerabilidad_urbana(inicio, fin, min_casos=20, min_defunciones=0):
    ''' Ajustar un modelo para los municipios urbanos y otro para los no urbanos '''
    inicio = pd.to_datetime(inicio, yearfirst=True)
    fin = pd.to_datetime(fin, yearfirst=True)
    fechas = pd.date_range(inicio, fin)
    resultados = []
    modelos = []
    
    asegura_archivos_covid_disponibles(fechas)
    f = IntProgress(min=0, max=len(fechas) - 1) # instantiate the bar
    display(f) # display the bar
    
    urbanos = municipios_urbanos()
        
    for count, fecha in enumerate(fechas):
        covid_municipal = tabla_covid_indicadores_municipales(fecha.strftime("%y%m%d"))
        covid_municipal = agregar_tasas_municipales(covid_municipal)
     
        caracteristicas = caracteristicas_modelos_municipios(covid_municipal)
        
        serie_urbanos = covid_municipal[covid_municipal.CLAVE_MUNICIPIO_RES.isin(urbanos.CLAVE_MUNICIPIO)]
            
        pls_urbanos = ajustar_pls_letalidad(serie_urbanos, caracteristicas,
                                            min_defunciones=min_defunciones, min_casos=min_casos)
        df_urbanos = calificar_municipios_letalidad_formato_largo(serie_urbanos, 
                                                                  pls_urbanos,
                                                                  caracteristicas,
                                                                  modelo='PLS_URBANO',
                                                                  dia_ajuste=fecha)
        resultados.append(df_urbanos)
        modelo = pd.DataFrame({'caracteristica': caracteristicas,
                               'coef': pls_urbanos.coef_})
        modelo['dia_ajuste'] = fecha
        modelo['modelo'] = 'PLS_URBANO'
        
        modelos.append(modelo)
        
        serie_no_urbanos = covid_municipal[~covid_municipal.CLAVE_MUNICIPIO_RES.isin(urbanos.CLAVE_MUNICIPIO)]
        pls_no_urbanos = ajustar_pls_letalidad(serie_no_urbanos, caracteristicas,
                                               min_defunciones=min_defunciones, min_casos=min_casos)
        df_no_urbanos = calificar_municipios_letalidad_formato_largo(serie_no_urbanos, 
                                                                  pls_no_urbanos,
                                                                  caracteristicas,
                                                                  modelo='PLS_NO_URBANO',
                                                                  dia_ajuste=fecha)
        resultados.append(df_no_urbanos)
        modelo = pd.DataFrame({'caracteristica': caracteristicas, 'coef': pls_no_urbanos.coef_})
        modelo['dia_ajuste'] = fecha
        modelo['modelo'] = 'PLS_NO_URBANO'

        modelos.append(modelo)
        
        f.value = count
    
    resultados_df = pd.concat(resultados, ignore_index=True)
    modelos_df = pd.concat(modelos, ignore_index=True)
    return modelos_df, resultados_df    

In [ ]:
pls, vulnerabilidad_urbana = calcular_vulnerabilidad_urbana('200620', '200621', min_defunciones=5, min_casos=0)

In [ ]:
pls.head()

## Postprocesamiento de resultados 

### agregar_vulnerabilidad_entidades

In [ ]:
#exports

def agregar_vulnerabilidad_entidades(serie_vulnerabilidad, campo_indice='valor'):
    df = serie_vulnerabilidad.copy()
    
    df['poblacion_entidad'] = df.groupby(['CLAVE_ENTIDAD_RES', 'dia_ajuste', 'modelo'])['pob2020'].transform('sum')
    df['proporcion_pob_entidad'] = df['pob2020'] / df['poblacion_entidad']
    
    df[f'{campo_indice}_entidad'] = df[campo_indice] * df['proporcion_pob_entidad']
    df[f'{campo_indice}_entidad'] = df.groupby(['CLAVE_ENTIDAD_RES', 'dia_ajuste', 'modelo'])[f'{campo_indice}_entidad'].transform('sum')
    df['letalidad_entidad'] = df['tasa_covid_letal'] * df['proporcion_pob_entidad']
    df['letalidad_entidad'] = df.groupby(['CLAVE_ENTIDAD_RES', 'dia_ajuste', 'modelo'])[f'letalidad_entidad'].transform('sum')

    
    return df

## Reporte de resultados

### guardar_resultados_csv

In [ ]:
#exports

def guardar_resultados_csv(resultados):
    reporte_csv = resultados.rename(columns={'tasa_covid_conocidos': 'casos_100k',
                             'tasa_covid_defun': 'defunciones_100k',
                             'tasa_covid_letal': 'letalidad',
                             'i_PLS_0': 'i_vulnerabilidad'})
    
    resultados_dir = 'resultados'
    sem_ent_cols = ['CLAVE_ENTIDAD_RES', 'nom_ent'] + ['i_vuln_estatal', 'semaforo_estado', 'semaforo_tend_estado']
    sem_mun_cols = ['CLAVE_MUNICIPIO_RES', 'nom_ent', 'nom_mun'] + ['i_vulnerabilidad', 'semaforo_municipio']

    nombre_csv_mun = os.path.join(resultados_dir, 'i_semaforo_municipios.csv')
    nombre_csv_ent = os.path.join(resultados_dir, 'i_semaforo_entidades.csv')
    nombre_csv_full = os.path.join(resultados_dir, 'semaforo_municipios.csv')

    reporte[sem_mun_cols].to_csv(nombre_csv_mun, index=False)
    reporte[sem_ent_cols].drop_duplicates().to_csv(nombre_csv_ent, index=False)
    reporte.drop(columns='geometry').to_csv(nombre_csv_full, index=False)


In [ ]:
guardar_resultados()

### mapas_serie_letalidad

In [ ]:
#exports

def mapas_serie_letalidad(serie_vulnerabilidad, gif=True, lapso_dias=1):
    '''
        Recibe el DataFrame con la vulnerabilidad calculada para un rango de fechas y genera los mapas de cada fecha. 
    '''
    dir_salida = 'resultados/mapas/letalidad'
    if not os.path.exists(dir_salida):
        os.mkdir(dir_salida)
        
    serie_vulnerabilidad = serie_vulnerabilidad[['dia_ajuste', 'geometry', 'tasa_covid_letal', 'CLAVE_MUNICIPIO_RES']].copy()
    serie_vulnerabilidad = serie_vulnerabilidad.drop_duplicates()
    
    inicio = serie_vulnerabilidad.dia_ajuste.min()
    fin = serie_vulnerabilidad.dia_ajuste.max()
    
    imagenes_mapas = []
    lista_mapas = []
    for fecha in pd.date_range(inicio, fin, freq=f'{lapso_dias}D'):
        datos_mapa = serie_vulnerabilidad[(serie_vulnerabilidad.dia_ajuste == fecha)]
        datos_mapa = datos_mapa[datos_mapa.tasa_covid_letal.notna()]
        fig = datos_mapa.plot(column='tasa_covid_letal', cmap='YlOrRd', scheme="equalinterval",
                        figsize=(10, 10), legend=True, k=10, edgecolor="face")
        # remove axis of chart
        fig.axis('off')

        # add a title
        fig.set_title('Letalidad COVID-19 Municipios de México', \
                  fontdict={'fontsize': '25', 'fontweight' : '3'})
        nombre_mapa = fecha.strftime("%y%m%d") + '_letalidad.jpg'
        ruta_archivo = os.path.join(dir_salida, nombre_mapa)
        lista_mapas.append(ruta_archivo)
        # position the annotation to the bottom left
        fig.annotate(fecha.strftime("%d/%m"),
                xy=(0.1, .225), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=35)
        
        chart = fig.get_figure()
        chart.savefig(ruta_archivo, dpi=300)
        plt.close(chart)
        imagenes_mapas.append(imageio.imread(ruta_archivo))

    inicio_str = inicio.strftime('%m%d')
    fin_str = fin.strftime('%m%d')
    imageio.mimsave(f'letalidad_covid_{inicio_str}_{fin_str}_{f"{lapso_dias}D"}.gif', imagenes_mapas, duration=1)
    optimize(f'letalidad_covid_{inicio_str}_{fin_str}_{f"{lapso_dias}D"}.gif')
    return lista_mapas
    


In [ ]:
mapas_serie_letalidad(serie_vulnerabilidad)

### mapas_serie_vulnerabilidad

In [ ]:
#exports

def mapas_serie_vulnerabilidad(serie_vulnerabilidad, modelo='PLS', gif=True, lapso_dias=1,
                               columna_vulnera='valor_tasa_covid_letal'):
    '''
        Recibe el DataFrame con la vulnerabilidad calculada para un rango de fechas y genera los mapas de cada fecha. 
    '''
    dir_salida = 'resultados/mapas/vulnerabilidad'
    if not os.path.exists(dir_salida):
        os.mkdir(dir_salida)
    
    serie_vulnerabilidad = serie_vulnerabilidad.copy()    
    inicio = serie_vulnerabilidad.dia_ajuste.min()
    fin = serie_vulnerabilidad.dia_ajuste.max()
    lista_mapas = []
    imagenes_mapas = []

    for fecha in pd.date_range(inicio, fin, freq=f'{lapso_dias}D'):
        datos_mapa = serie_vulnerabilidad[(serie_vulnerabilidad.dia_ajuste == fecha) &
                                         (serie_vulnerabilidad.modelo == modelo)]
        datos_mapa = datos_mapa[datos_mapa[columna_vulnera].notna()]
        fig = datos_mapa.plot(column=columna_vulnera, cmap='RdYlBu_r', scheme="Quantiles",
                        figsize=(10, 10), legend=True, k=5, edgecolor="face")
        # remove axis of chart
        fig.axis('off')

        # add a title
        fig.set_title('Vulnerabilidad COVID-19 Municipios de México', \
                  fontdict={'fontsize': '25', 'fontweight' : '3'})
        nombre_mapa = fecha.strftime("%y_%m_%d") + '_vulnerabilidad.jpg'
        ruta_archivo = os.path.join(dir_salida, nombre_mapa)
        lista_mapas.append(ruta_archivo)
        
        # position the annotation to the bottom left
        fig.annotate(fecha.strftime("%d/%m"),
                xy=(0.1, .225), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                fontsize=35)
        
        chart = fig.get_figure()
        chart.savefig(ruta_archivo, dpi=300)
        imagenes_mapas.append(imageio.imread(ruta_archivo))

        plt.close(chart)
    inicio_str = inicio.strftime('%m%d')
    fin_str = fin.strftime('%m%d')
    imageio.mimsave(f'vulnerabilidad_covid_{inicio_str}_{fin_str}_{f"{lapso_dias}D"}.gif', imagenes_mapas, duration=1)
    optimize(f'vulnerabilidad_covid_{inicio_str}_{fin_str}_{f"{lapso_dias}D"}.gif')
                    
    return lista_mapas

    
    

In [ ]:
mapas_serie_vulnerabilidad(serie_vulnerabilidad)

### guardar_shape_vulnerabilidad

In [ ]:
#exports
def guardar_shape_vulnerabilidad(vulnerabilidad_gdf, nombre_archivo, fecha_ajuste):
    fecha_ajuste = pd.to_datetime(fecha_ajuste, yearfirst=True)
    
    vulnerabilidad_dia = vulnerabilidad_gdf[vulnerabilidad_gdf.dia_ajuste == fecha_ajuste].copy()
    vulnerabilidad_dia.dia_ajuste = vulnerabilidad_dia.dia_ajuste.dt.strftime("%d-%m-%y")
    vulnerabilidad_dia.to_file(nombre_archivo)

In [ ]:
guardar_shape_vulnerabilidad(serie_vulnerabilidad, 'prueba_vulnerabilidad.shp', fecha_ajuste='200427')

### guardar_serie_vulnerabilidad

In [ ]:
#exports

def checkpoint_vulnerabilidad(serie_vulnerabilidad, nombre_archivo):
    serie_vulnerabilidad.drop(columns='geometry').to_feather(nombre_archivo)

In [ ]:
#exports

def cargar_checkpoint_vulnerabilidad(nombre_archivo):
    vulnerabilidad_df = pd.read_feather(nombre_archivo)
    geometria_municipal = gpd.read_file('datos/municipios/marco_2019.json')
    
    vulnerabilidad_df= vulnerabilidad_df.merge(geometria_municipal[['geometry', 'municipio_cvegeo']],
                            left_on=['CLAVE_MUNICIPIO_RES'], right_on='municipio_cvegeo')
    
    return vulnerabilidad_df


In [ ]:
vulnerabilidad_df = cargar_checkpoint_vulnerabilidad('prueba.feather')

In [ ]:
null_geoms = vulnerabilidad_df.loc[vulnerabilidad_df.geometry.isna()]

In [ ]:
null_geoms[['ENTIDAD_RES', 'MUNICIPIO_RES']].drop_duplicates()

,ENTIDAD_RES,MUNICIPIO_RES
258816,OAXACA,ASUNCIÓN OCOTLÁN
265984,OAXACA,GUELATAO DE JUÁREZ
272896,OAXACA,NATIVIDAD
297216,OAXACA,SAN JACINTO AMILPAS
306176,OAXACA,SAN JUAN CHILATECA
432896,PUEBLA,RAFAEL LARA GRAJALES
436992,PUEBLA,SAN MARTÍN TOTOLTEPEC


In [ ]:
from nbdev.export import notebook2script
notebook2script()